In [1]:
import numpy as np
import pptk
import networkx as nx
import imageio
import glob
import re
from random import sample
from scipy.spatial import distance
from time import strftime
import os
import scipy.optimize as optimize
import math
from sklearn.linear_model import LinearRegression
import time
from moving_least_square import *
from tempfile import TemporaryFile

In [2]:
if os.name == 'nt': # Windows
    system_win = 1
else:
    system_win = 0

In [3]:
# given the data points, return the MST of the point cloud generated from the PNG files and an array 
# of points positions
def getMSTFromDataPoint(data, drawMST: bool, sampleNumber: int=5000):
    # Read points data from PNGs 
    if(sampleNumber > len(data)):
        sampleNumber = len(data)
        
    # default sample 5000 points from the whole set, otherwise it would take too long
    print("---------------")
    print("There are " + str(len(data)) + " points in total. Now sampleling " + str(sampleNumber) + " points from them")
    sample_data = np.asarray(sample(list(data), sampleNumber))
    print("---------------")
    print("Done!")
    #display the points 
    displayPoints(sample_data, 1.3)
    #Create a networkX graph instance that represent MST
    print("---------------")
    print("Begin creating a MST of the sampled points cloud")
    MST = CreateMSTGraph(sample_data)
    print("---------------")
    print("MST creation Done!")
    if(drawMST):
        nx.draw(MST, dict(enumerate(sample_data[:, :2])))
    return (MST, sample_data)
    

In [4]:
def displayPoints(data, pointSize):
    v = pptk.viewer(data)
    v.set(point_size=pointSize)

In [5]:
def readPointFromTXT(filepath):
    data = np.genfromtxt(fname=filepath, skip_header=0)
    return data

In [6]:
def ReadPointFromPNG(filepath):
    path_list = [im_path for im_path in glob.glob(filepath)]
    if system_win:
        path_list_parsed = [re.split('\\\\|\.', path) for path in path_list]
    else:
        path_list_parsed = [re.split('/|\.', path) for path in path_list]
    path_list_parsed_valid = [x for x in path_list_parsed if x[-1] == 'png']
    path_list_parsed_valid = sorted(path_list_parsed_valid, key=lambda x:int(x[-2]))
    data_valid = []
    delta = 0.5
    thickness = len(path_list_parsed_valid) * 3 * delta
    for path in path_list_parsed_valid:
        s = ""
        if system_win:
            s = "\\"
        else:
            s = "/"
        s = s.join(path)
        s = s[:-4] + '.png'
        image = imageio.imread(s)
        for row in range(len(image)):
            for col in range(len(image)):
                if image[row][col] > 100:
                    data_valid.append([row, col, thickness])
        thickness -= 3*delta
    return data_valid

In [7]:
# This function is used to limited the number of edges in the original graph.
# Instead of creating a graph with full connectivity, this function will return 
# a list of neighbor points for each point and we will only connect them in the graph
def getNearbyPoints(pointsData):
    D = distance.squareform(distance.pdist(pointsData))
    closestIndicies = np.argsort(D, axis=1)
    closestDis = np.sort(D, 1)
    threshold = 10 # This number can be changed. The greater this number, the more edges
    return (closestIndicies[:, 1:threshold], closestDis[:, 1:threshold])

In [8]:
def CreateMSTGraph(pointsData):
    print("---------------")
    print("Begin calculating nearby points for each point")
    nearbyInfo = getNearbyPoints(pointsData)
    print("---------------")
    print("Nearby points calculation Done!")
    print("---------------")
    print("Begin construct graph")
    G=nx.Graph()
    closestIndicies = nearbyInfo[0]
    closestDis = nearbyInfo[1]
    for firstPIndex in range(len(closestIndicies)):
        for second in range(len(closestIndicies[firstPIndex])):
            secondPIndex = closestIndicies[firstPIndex][second]
            G.add_edge(firstPIndex, secondPIndex , weight = closestDis[firstPIndex][second])
    print("---------------")
    print("Graph construction Done!")
    print("---------------")
    print("Begin calculate MST")
    G = nx.minimum_spanning_tree(G)
    print("---------------")
    print("MST calculation Done!")
    return G

In [9]:
# Impliment the collect algorithm for 3D points in the paper
def collectPoints1(P: int, PStar: int):
    global H_glo
    global graph
    global pointsCor3D
    global A

    A.append(P)
    for edge in graph.edges(P):
        Pj = edge[1]
        if(Pj) not in A and distance.euclidean(pointsCor3D[Pj], pointsCor3D[PStar]) < H_glo:
            collectPoints1(Pj, PStar)

In [10]:
def collectPointsNonrec(PStar: int):
    global H_glo
    global graph
    global pointsCor3D
    global distance_dict 
    global dirty_dict
    toExplore = [PStar]
    A = [PStar]
    distance_dict[(PStar, PStar)] = 0
    weight_dict[((PStar, PStar))] = 1
    
    while len(toExplore) > 0:
        curP = toExplore[0]
        del toExplore[0]
        for edge in graph.edges(curP):
            Pj = edge[1]
            if(Pj) not in A:
                if (Pj, PStar) not in distance_dict or (PStar, Pj) not in distance_dict or \
                dirty_dict[PStar] == 1 or dirty_dict[Pj] == 1:
                    dist_temp = distance.euclidean(pointsCor3D[Pj], pointsCor3D[PStar])
                    distance_dict[(Pj, PStar)] = dist_temp
                    distance_dict[(PStar, Pj)] = dist_temp
                    weight_dict[((PStar, Pj))] = weightFun(PStar, Pj)
                    weight_dict[((Pj, PStar))] = weightFun(PStar, Pj)
                    dirty_dict[Pj] = 0
                    dirty_dict[PStar] = 0
                    
                if distance_dict[(Pj, PStar)] < H_glo:
                    toExplore.append(Pj)
                    A.append(Pj)
    return A

In [11]:
def weightFun(P1, P2):
    global distance_dict 
    global dirty_dict
    global pointsCor3D
    if(P1 == P2):
        return 1
    return math.exp(-1 * (distance_dict[(P1, P2)]**2)/(H_glo**2))

In [12]:
def calculateRegressionPlane(PStar, A: list):
    global pointsCor3D
    global H_glo
    global weight_dict
    
    '''
    for point in A:
        if (PStar, point) not in weight_dict or dirty_dict[point] == 1 or dirty_dict[PStar]:
            weight_dict[((PStar, point))] = weightFun(PStar, point)
            weight_dict[((point, PStar))] = weightFun(PStar, point)
    '''
    
    def f(params):
        a, b, c = params 
        loss = 0
        for point in A:
            point_cor = pointsCor3D[point]
            loss += ((a*point_cor[0] + b*point_cor[1] + c - point_cor[2])**2)*weight_dict[((point, PStar))]
            #loss += ((a*point_cor[0] + b*point_cor[1] + c - point_cor[2])**2)*weightFun(PStar, point)
        return loss
    
    initial_guess = [1, 1, 1]
    result = optimize.minimize(f, initial_guess, method = 'Nelder-Mead')
    if result.success:
        fitted_params = result.x
    else:
        raise ValueError(result.message)
    return fitted_params

In [13]:
def projectPoints(params, A: list):
    global pointsCor3D
    a, b, c = params
    normal = np.asarray([a, b, -1])
    normal = normal / np.linalg.norm(normal)
    pointOnPlane = np.asarray([0, 0, c])
    projectionPointsCor = []
    for point in A:
        point_cor = np.asarray(pointsCor3D[point])
        pointToPlaneV = point_cor - pointOnPlane
        dist = np.dot(normal, pointToPlaneV)
        projectionPointcor = point_cor - dist*normal
        projectionPointsCor.append(list(projectionPointcor))
    return projectionPointsCor

In [14]:
# this function converted the 3D coordinate system of points in a plane to 2D, returns a list of new coordinates
# each of them also has x, y and z component but z is equal to 0
# this finction also will return the info of the plane, which can be used to convert a 2D coordinate to 3D again
# The format of the plane info is [u, v, origin] (u is a unit vector in 3D representing plane's x axis, y is a unit 
# vector in 3D representing plane's y axis, origin is a coordinate in 3D of plane's origin )
def convertTo2DCor(pointsCor, planeParam):
    a, b, c = planeParam
    origin = np.array([0, 0, c])
    u = np.array([0, 0, c]) - np.array([1, 1, a + b + c])
    u = u / np.linalg.norm(u)
    normal = np.array([a, b, -1])
    v = np.cross(u, normal)
    v = v / np.linalg.norm(v)
    convertedPointsCor = []
    for pointCor in pointsCor:
        oriV = np.array(pointCor) - origin
        new_x = np.dot(oriV, u)
        new_y = np.dot(oriV, v)
        convertedPointsCor.append([new_x, new_y, 0])
    planeInfo = [u, v, origin]
    return (convertedPointsCor, planeInfo)
    

In [15]:
# return the 2D coordinate of the 3D points and the information of the regression plane, which the points are located
# require gloable perameters graph and pointsCor3D
def get2DCorFrom3D(targetPoint):
    global graph
    global pointsCor3D
    '''
    global A
    A = []
    start = time.time()
    collectPoints1(targetPoint, targetPoint)
    end = time.time()
    print("collectPoints1:")
    print(start - end)
    '''
    #start = time.time()
    localPoints = collectPointsNonrec(targetPoint)
    #end = time.time()
    #print("collectPointsNonrec:")
    #print(start - end)
    #print(localPoints)
    #displayPoints(pointsCor3D[np.asarray(localPoints)], 0.5)
    #start = time.time()
    params = calculateRegressionPlane(targetPoint, localPoints)
    #print(params)
    #end = time.time()
    #print("calculateRegressionPlane:")
    #print(start - end)
    
    '''
    xyz = []
    a, b, c = params
    for x in range(0, 10):
        for y in range(0, 10):
            z = a*x + b*y + c
            xyz.append([x, y, z])
    displayPoints(xyz, 0.5)
    '''          
    #start = time.time()
    projectionPointsCor = projectPoints(params, localPoints)
    #end = time.time()
    #print("projectPoints:")
    #print(start - end)
    #start = time.time()
    points2DCor, planeInfo = convertTo2DCor(projectionPointsCor, params)
    #end = time.time()
    #print("convertTo2DCor:")
    #print(start - end)
    return (points2DCor, planeInfo)

In [16]:
# This function takes a single point's 2D coordinate and transform it into 3D base on the planeInfo
def get3DCorFrom2D(pointCor, planeInfo):
    u, v, origin = planeInfo
    vectorElem1 = pointCor[0]*u
    vectorElem2 = pointCor[1]*v
    newCor = vectorElem1 + vectorElem2 + origin
    return newCor

In [17]:
#compue the line regression
def calculateRegressionLine(pointsCor):
    X = np.array([x[0] for x in pointsCor]).reshape(-1, 1)
    Y = np.array([x[1] for x in pointsCor]).reshape(-1, 1)
    linear_regressor = LinearRegression()  # create object for the class
    linear_regressor.fit(X, Y)  # perform linear regression
    return(linear_regressor.coef_[0], linear_regressor.intercept_[0])

In [18]:
def rotatePointsCor(pointsCor, lineCoef):
    pointsCor = np.array(pointsCor)
    theta = math.atan(lineCoef)
    c, s = math.cos(theta), math.sin(theta)
    R = np.array([(c,-s, 1), (s, c, 1)])
    newPointsCor = []
    for point in pointsCor:
        newPointsCor.append(R.dot(point))
    return np.asarray(newPointsCor)

In [19]:
#filePath = "mri_label_2016/*.png"
#pointData = ReadPointFromPNG(filePath)

In [20]:
#np.save('./temp', pointData)

In [21]:
pointData = np.load('./temp.npy')

In [22]:
pointData

array([[122., 222., 195.],
       [122., 223., 195.],
       [122., 224., 195.],
       ...,
       [176., 418.,  54.],
       [176., 419.,  54.],
       [177., 417.,  54.]])

In [23]:
(graph, pointsCor3D) = getMSTFromDataPoint(pointData, drawMST=True)

---------------
There are 818757 points in total. Now sampleling 5000 points from them
---------------
Done!
---------------
Begin creating a MST of the sampled points cloud
---------------
Begin calculating nearby points for each point
---------------
Nearby points calculation Done!
---------------
Begin construct graph
---------------
Graph construction Done!
---------------
Begin calculate MST
---------------
MST calculation Done!
---------------
MST creation Done!


In [ ]:
H_ini = 10
H_delta = 30
H_glo = 0
trial_limit =3
correlation_threshold = 0.4
weight_dict = {}
distance_dict = {}
dirty_dict = {}
for point in range(len(pointsCor3D)):
    dirty_dict[point] = 0
results = []
for targetPoint in range(len(pointsCor3D)):
    cur_correlation = 0
    H_glo = H_ini
    trial = 0
    correlation_hist = []
    planeInfo = 0
    
    while(cur_correlation < correlation_threshold and trial < trial_limit):
        localPointsCor2D, planeInfo = get2DCorFrom3D(targetPoint)
        #displayPoints(localPointsCor2D, 0.5)
        slope, intersept = calculateRegressionLine(localPointsCor2D)
        rotatedPointsCor = rotatePointsCor(localPointsCor2D, slope)
        #displayPoints(np.insert(rotatedPointsCor, 2, values=0, axis=1), 0.5)
        cur_correlation = abs(np.corrcoef(rotatedPointsCor[:, 0],rotatedPointsCor[:, 1])[0][1])
        print(targetPoint, "trial",str(trial), "H =", H_glo, ":" , str(cur_correlation))
        H_glo += H_delta * (trial + 1)
        trial += 1
        correlation_hist.append(cur_correlation)
        
    #print(pointsCor3D[targetPoint][:2])
    #print(localPointsCor2D)
    centerPoint2D = np.asarray(localPointsCor2D)[:1, :2]
    #print(centerPoint2D)
    #print(localPointsCor2D)
    
    newCor = Moving_Least_Square(centerPoint2D[0], np.asarray(localPointsCor2D)[:, :2])
    print(centerPoint2D[0], newCor)
    newCor3D = get3DCorFrom2D(newCor, planeInfo)
    print(pointsCor3D[targetPoint], newCor3D)
    pointsCor3D[targetPoint] = list(newCor3D)
    dirty_dict[targetPoint] = 1
    #if(targetPoint%100 == 0):
        #displayPoints(pointsCor3D, 0.5)
    results.append(max(correlation_hist))
displayPoints(pointsCor3D, 0.5)    

0 trial 0 H = 10 : 0.4221534417877219
[-332.00227876 -221.29082127] [-332.79552959 -221.292808  ]
[ 79.  391.  106.5] [ 79.54199176 391.57067088 106.75779323]
1 trial 0 H = 10 : 0.024666376897701563
1 trial 1 H = 40 : 0.13031046621618095
1 trial 2 H = 100 : 0.467213413726652
[-281.31768701  124.17239164] [-269.77403555  124.02576641]
[285.  114.  160.5] [276.29131905 102.56554148 148.13312669]
2 trial 0 H = 10 : 0.06811041966076696
2 trial 1 H = 40 : 0.3150807211269823
2 trial 2 H = 100 : 0.06243770664380222
[-306.78397639  132.41049978] [-280.66409955  132.1950428 ]
[307. 121. 141.] [288.86462007 103.25726167 147.23546303]
3 trial 0 H = 10 : 0.36834561655383075
3 trial 1 H = 40 : 0.013966227502503861
3 trial 2 H = 100 : 0.17665249852834683
[-288.00423484  -96.00322249] [-288.60318787  -95.99517906]
[135.  271.  181.5] [135.57284953 270.6406446  175.04033861]
4 trial 0 H = 10 : 0.19888773704764254
4 trial 1 H = 40 : 0.40611841642229785
[-245.12265531   31.71212137] [-236.89604107   34.

39 trial 2 H = 100 : 0.32099499943940407
[-467.37656142   83.60802153] [-468.16190665   83.60955818]
[389. 272. 156.] [389.61203397 272.49715007 156.57444532]
40 trial 0 H = 10 : 0.1277580551256562
40 trial 1 H = 40 : 0.3830215396815839
40 trial 2 H = 100 : 0.7675529506802841
[-422.61423709 -192.99592476] [-413.93838469 -192.97143224]
[173. 408.  87.] [160.38303591 412.29355807 116.45982928]
41 trial 0 H = 10 : 0.018549611437959253
41 trial 1 H = 40 : 0.12242948592168192
41 trial 2 H = 100 : 0.6927954078841025
[-500.16439932   29.84071142] [-513.4462551    29.86631967]
[364.  323.  169.5] [375.33363441 333.2086548  162.06388165]
42 trial 0 H = 10 : 0.008859759690613185
42 trial 1 H = 40 : 0.5582978581056505
[-492.65938059 -174.8066403 ] [-492.5383467  -174.80618873]
[186. 433. 102.] [185.4922137  432.55172044 103.00506509]
43 trial 0 H = 10 : 0.0460152900633659
43 trial 1 H = 40 : 0.09792321897772097
43 trial 2 H = 100 : 0.0817266899951716
[-225.22185391   59.05301221] [-221.91358377  

81 trial 0 H = 10 : 0.5702531216409921
[-283.89905728   39.76607676] [-281.9278846    39.79207236]
[179. 123. 171.] [178.12581648 122.05041173 172.52535222]
82 trial 0 H = 10 : 0.12836469661469424
82 trial 1 H = 40 : 0.10739611141567765
82 trial 2 H = 100 : 0.6431499940956201
[-294.2749465  -150.89691147] [-300.88829468 -150.85449705]
[ 99. 310. 165.] [103.65267548 315.15953003 167.19981832]
83 trial 0 H = 10 : 0.06170300416947835
83 trial 1 H = 40 : 0.14131306050536785
83 trial 2 H = 100 : 0.015534481869925914
[-289.4494269  -182.46130785] [-310.95191492 -181.72639331]
[ 75.  328.  154.5] [ 90.24925234 344.17134738 161.47255344]
84 trial 0 H = 10 : 0.4611188844201476
[-305.19786939  -61.94046623] [-304.02537645  -61.90674805]
[128. 213. 189.] [128.84600664 212.90180649 186.58953878]
85 trial 0 H = 10 : 0.24557772311840395
85 trial 1 H = 40 : 0.12345389742765332
85 trial 2 H = 100 : 0.08522041330386457
[-215.81697676  -49.08046874] [-226.59911891  -48.73063718]
[118.  187.  175.5] [126

122 trial 1 H = 40 : 0.08357594215296425
122 trial 2 H = 100 : 0.13680085941732342
[-206.16931472   81.40125547] [-219.9128581    77.35426884]
[205.   87.  181.5] [209.41379532 100.82006362 159.82425398]
123 trial 0 H = 10 : 0.04794533723619657
123 trial 1 H = 40 : 0.3305701457247926
123 trial 2 H = 100 : 0.7613943657683281
[-528.98751834    8.7797818 ] [-529.3331266     8.78087834]
[364.  351.  157.5] [363.10503831 350.79266602 161.14546946]
124 trial 0 H = 10 : 0.014455582122455669
124 trial 1 H = 40 : 0.28972382673356056
124 trial 2 H = 100 : 0.41287524442841245
[-454.06913097 -231.26517504] [-446.85099694 -231.24069153]
[107. 408. 117.] [102.47384752 403.66889143 120.60932315]
125 trial 0 H = 10 : 0.2419996478041771
125 trial 1 H = 40 : 0.11335945195361705
125 trial 2 H = 100 : 0.13528112959057637
[-288.79779733  -90.33042824] [-285.58242842  -90.3760243 ]
[140. 266. 168.] [137.50443789 264.66291353 175.81768964]
126 trial 0 H = 10 : 0.08003058730984562
126 trial 1 H = 40 : 0.16963

161 trial 1 H = 40 : 0.2537663821575329
161 trial 2 H = 100 : 0.18054652898570187
[-200.10720539   99.71687735] [-224.36661505   99.22635534]
[212.   71.  163.5] [228.24918529  89.06422068 157.37028981]
162 trial 0 H = 10 : 0.34618873509906534
162 trial 1 H = 40 : 0.5513644395386443
[-505.11411479 -116.99066289] [-500.90683013 -116.98335119]
[275. 439. 147.] [271.76493051 436.36021718 151.27560375]
163 trial 0 H = 10 : 0.799981348848648
[-502.12800892  -14.01985987] [-502.09492327  -14.01991366]
[332.  351.  139.5] [331.34206037 350.84301471 141.34050988]
164 trial 0 H = 10 : 0.35053471601644554
164 trial 1 H = 40 : 0.18406822570473005
164 trial 2 H = 100 : 0.12103456934936102
[-255.99389155  -47.08201511] [-242.03267098  -47.24539687]
[149. 213. 180.] [137.70276742 203.57163842 174.87132435]
165 trial 0 H = 10 : 0.265121819753648
165 trial 1 H = 40 : 0.1509326934866432
165 trial 2 H = 100 : 0.29345217308642924
[-225.53442504    4.18080122] [-225.53377894    4.18079308]
[161. 156. 165.

...

363 trial 1 H = 40 : 0.36304594651068156
363 trial 2 H = 100 : 0.04391026574839776
[-281.6803456   -81.40602811] [-282.42302346  -81.39421963]
[141.  257.  187.5] [141.79197328 256.4369211  176.96599358]
364 trial 0 H = 10 : 0.14234166030029335
364 trial 1 H = 40 : 0.012327777677560301
364 trial 2 H = 100 : 0.18184397312974016
[-286.95692364  -92.7061659 ] [-286.81151798  -92.70803798]
[137.  267.  172.5] [136.80917587 267.24353315 175.50078313]
365 trial 0 H = 10 : 0.5692210983575471
[-340.78018821  146.65595836] [-341.83512292  146.64293811]
[313.  149.  133.5] [313.65695345 149.02831625 134.61359005]
366 trial 0 H = 10 : 0.9728439705693136
[-438.41887578  110.0073576 ] [-438.62401866  110.00477518]
[359.  226.  157.5] [359.17394008 225.72937264 158.06382106]
367 trial 0 H = 10 : 0.46912067378071226
[-264.45832201   50.8771307 ] [-262.70801792   50.83215183]
[223.  151.  160.5] [221.66989275 149.85252052 162.40117913]
368 trial 0 H = 10 : 0.07946387024216318
368 trial 1 H = 40 : 0.42

403 trial 1 H = 40 : 0.556573850357615
[-389.37706319 -185.14033702] [-391.78147434 -185.1473219 ]
[143.  405.   79.5] [145.32008271 406.99268059  85.94211919]
404 trial 0 H = 10 : 0.06621360531976729
404 trial 1 H = 40 : 0.29868437543423715
404 trial 2 H = 100 : 0.5180842876850441
[-257.79792302  118.00259208] [-263.52389172  117.61308485]
[261.   95.  145.5] [265.73729495  99.73611109 150.64655014]
405 trial 0 H = 10 : 0.17100256821503226
405 trial 1 H = 40 : 0.5035243751938592
[-389.11023918 -191.16927953] [-386.5675112  -191.16152294]
[140.  410.   88.5] [138.10330789 408.17919513  87.29149122]
406 trial 0 H = 10 : 0.7095067022760474
[-457.36642219 -171.31789173] [-455.16714852 -171.32607089]
[199. 434. 105.] [197.414655   432.59467077 104.22871931]
407 trial 0 H = 10 : 0.943989734248079
[-342.60994144  152.40102185] [-342.60995778  152.40102158]
[317.  145.  139.5] [316.99616307 145.03742162 139.46955692]
408 trial 0 H = 10 : 0.2845437285281678
408 trial 1 H = 40 : 0.0530668962668

447 trial 1 H = 40 : 0.2938898362242396
447 trial 2 H = 100 : 0.19651764991985118
[-228.99757276  105.5147456 ] [-237.13023705  105.34235386]
[235.  86. 159.] [240.32730344  91.79391925 156.09727043]
448 trial 0 H = 10 : 0.5681542918707976
[-319.01132013   82.81690805] [-316.01777311   82.78278035]
[257. 163. 117.] [254.97089943 160.95238189 117.81062893]
449 trial 0 H = 10 : 0.028696268208882804
449 trial 1 H = 40 : 0.14628881351920647
449 trial 2 H = 100 : 0.7845809554408446
[-518.26700947   12.6440206 ] [-513.47282131   12.63110319]
[369.  351.  160.5] [365.35420684 347.60201489 161.02628266]
450 trial 0 H = 10 : 0.477463835999944
[-198.74882946  -11.70064601] [-199.31408774  -11.65864156]
[130.  146.  175.5] [130.52867596 146.34092355 175.67916101]
451 trial 0 H = 10 : 0.021870129399299675
451 trial 1 H = 40 : 0.30274602189021166
451 trial 2 H = 100 : 0.03989400940018503
[-311.82693913  145.72166695] [-288.51635494  145.54544065]
[321. 119. 156.] [303.84175039  99.61176117 147.5764

490 trial 1 H = 40 : 0.3396053440961941
490 trial 2 H = 100 : 0.5061896735432064
[-279.63680619  -53.02486714] [-255.13099613  -53.7414021 ]
[160.  235.  181.5] [142.25559247 217.9752977  180.08115162]
491 trial 0 H = 10 : 0.7265815551652379
[-265.50097056   42.79876629] [-266.7887494    42.80274042]
[164.  115.  181.5] [164.9299782  115.70748671 182.10256541]
492 trial 0 H = 10 : 0.023406390053971174
492 trial 1 H = 40 : 0.6539437824971479
[-508.24512332  -28.63675863] [-517.10978969  -28.61914512]
[323.  364.  151.5] [330.57144215 370.9528258  148.08689831]
493 trial 0 H = 10 : 0.46201797624005375
[-232.26966139 -360.05993607] [-229.43537925 -360.05427764]
[123.  401.   91.5] [121.53877018 399.40153909  93.33179878]
494 trial 0 H = 10 : 1.0
[-333.00816959  132.95907817] [-333.00816172  132.95907812]
[298. 148. 135.] [297.83132309 149.2774315  133.94082284]
495 trial 0 H = 10 : 0.8817825360912352
[-304.0585363  -335.80403411] [-306.52445026 -335.8073224 ]
[144.  401.  154.5] [144.3090

533 trial 2 H = 100 : 0.34997366119568196
[-278.11916939   96.9481408 ] [-253.39559934   98.89838202]
[260.  123.  145.5] [244.76570186 104.9281134  153.5653404 ]
534 trial 0 H = 10 : 0.1847858289294356
534 trial 1 H = 40 : 0.057474585775808745
534 trial 2 H = 100 : 0.10224581877731807
[-249.94152611   51.71943026] [-225.83288117   50.59871927]
[212.  138.  166.5] [192.77249264 121.4685841  161.23947907]
535 trial 0 H = 10 : 0.9876264789245465
[-240.65815911 -287.04581266] [-240.65818901 -287.04581259]
[ 66. 330. 165.] [ 65.98159639 330.0129744  164.98139301]
536 trial 0 H = 10 : 0.9999999999999999
[-263.36472717 -153.89957056] [-263.36471667 -153.89957058]
[117. 227. 168.] [116.99996169 227.00002886 167.99997104]
537 trial 0 H = 10 : 0.45600827017007367
[-216.46431891  -90.92564424] [-214.56962011  -90.89530407]
[ 94.  210.  178.5] [ 92.16284263 209.24641153 176.67068859]
538 trial 0 H = 10 : 0.9999999999999999
[  50.71193517 -270.82009745] [  50.71192978 -270.82009747]
[103.  180.  1

In [ ]:
[-546.9247741   -99.82680608] [-540.85305533  -99.82008771]
[288. 429. 165.] [286.88648168 428.04628798 151.69990822]